In [1]:
import import_ipynb
import numpy as np
from data_tools import *
from args import parser
import librosa
import os
import soundfile as sf

importing Jupyter notebook from data_tools.ipynb
importing Jupyter notebook from args.ipynb


In [2]:
args, unknown = parser.parse_known_args()

In [3]:
#folder containing noises
train_noise_dir = args.train_noise_dir     
#folder containing clean voices
train_voice_dir = args.train_voice_dir

test_noise_dir = args.test_noise_dir     
#folder containing clean voices
test_voice_dir = args.test_voice_dir

#path to save time series
path_save_time_series = args.path_save_time_series
#path to save sounds
path_save_sound = args.path_save_sound
#path to save spectrograms
path_save_spectrogram = args.path_save_spectrogram  
# Sample rate to read audio
sample_rate = args.sample_rate
# Minimum duration of audio files to consider
min_duration = args.min_duration   
#Frame length for training data
frame_length = args.frame_length   
# hop length for clean voice files
hop_length_frame = args.hop_length_frame       
# hop length for noise files
hop_length_frame_noise = args.hop_length_frame_noise
# How much frame to create for training
nb_samples = args.nb_samples
#nb of points for fft(for spectrogram computation)
n_fft = args.n_fft  
#hop length for fft
hop_length_fft = args.hop_length_fft
SNR=[-10,-5,0,5,10]
test_out= args.audio_dir_prediction

In [4]:
list_train_noise_files = os.listdir(train_noise_dir)
list_train_voice_files = os.listdir(train_voice_dir)
list_test_noise_files = os.listdir(test_noise_dir)
list_test_voice_files = os.listdir(test_voice_dir)

In [5]:
def remove_ds_store(lst):
    #remove mac specific file if present
    if '.DS_Store' in lst:
        lst.remove('.DS_Store')
    return lst

In [6]:
#Remove .DS_Store extension files
list_train_noise_files = remove_ds_store(list_train_noise_files)
list_train_voice_files = remove_ds_store(list_train_voice_files)
list_test_noise_files = remove_ds_store(list_test_noise_files)
list_test_voice_files = remove_ds_store(list_test_voice_files)

In [7]:
#Length of the noise and voice files
nb_train_voice_files = len(list_train_voice_files)
nb_train_noise_files = len(list_train_noise_files)
nb_test_voice_files = len(list_test_voice_files)
nb_test_noise_files = len(list_test_noise_files)
len_SNR= len(SNR)

In [8]:
list_train_noise_files= sorted_nicely(list_train_noise_files)
list_train_voice_files= sorted_nicely(list_train_voice_files)
list_test_noise_files= sorted_nicely(list_test_noise_files)
list_test_voice_files= sorted_nicely(list_test_voice_files)

In [9]:
#Length of the noise and voice files
nb_train_voice_files = len(list_train_voice_files)
nb_train_noise_files = len(list_train_noise_files)
nb_test_voice_files = len(list_test_voice_files)
nb_test_noise_files = len(list_test_noise_files)
len_SNR= len(SNR)

In [10]:
nb_train_voice_files, nb_train_noise_files, nb_test_voice_files, nb_test_noise_files, len_SNR

(20, 10, 20, 10, 5)

In [11]:
frame_length, hop_length_frame, hop_length_frame_noise

(8064, 8064, 8064)

In [12]:
# Extracting noise and voice from folder and convert to numpy
noise = audio_files_to_numpy(train_noise_dir, list_train_noise_files, sample_rate, frame_length, hop_length_frame_noise, min_duration)
voice = audio_files_to_numpy(train_voice_dir, list_train_voice_files, sample_rate, frame_length, hop_length_frame, min_duration)

In [13]:
voice.shape, noise.shape

((12000, 8064), (12000, 8064))

In [14]:
train_voice= voice[0:noise.shape[0], :]
#test_voice= voice[10020:10080,:]

train_noise= noise[0:noise.shape[0], :]
#test_noise= noise[10020:10080, :]

In [15]:
train_voice.shape, train_noise.shape
#test_voice.shape, test_noise.shape

((12000, 8064), (12000, 8064))

In [16]:
# Squared spectrogram dimensions
dim_square_spec = int(n_fft / 2) + 1 
dim_square_spec

128

In [17]:
nb_samples= train_voice.shape[0]
nb_samples

12000

# Train data creation

In [18]:
for i in range (len_SNR):
    prod_voice, prod_noise, prod_noisy = blend_noise_randomly(train_voice, train_noise, nb_samples, frame_length, SNR[i])
    
#     voice= prod_voice.reshape(1, nb_samples*frame_length)
#     noise= prod_noise.reshape(1, nb_samples*frame_length)
#     noisy= prod_noisy.reshape(1, nb_samples*frame_length)
#     sf.write('./Train/sound/male/male= '+ str(SNR[i])+'.wav', voice[0, :], samplerate= sample_rate)
#     sf.write('./Train/sound/female/female= '+ str(SNR[i])+'.wav', noise[0, :], samplerate= sample_rate)
#     sf.write('./Train/sound/mixed/mixed= '+ str(SNR[i])+'.wav', noisy[0, :], samplerate= sample_rate)
    
    
    voice_complex_mat, voice_real, voice_imag= numpy_audio_to_matrix_spectrogram(prod_voice, dim_square_spec, n_fft, hop_length_fft)
    del prod_voice, voice_complex_mat
    
    noise_complex_mat, noise_real, noise_imag= numpy_audio_to_matrix_spectrogram(prod_noise, dim_square_spec, n_fft, hop_length_fft)
    del prod_noise, noise_complex_mat
    
    noisy_complex_mat, noisy_real, noisy_imag= numpy_audio_to_matrix_spectrogram(prod_noisy, dim_square_spec, n_fft, hop_length_fft)
    del prod_noisy, noisy_complex_mat
    
    
    voice_concat= np.concatenate((voice_real, voice_imag), axis=0)
    del voice_real, voice_imag
    
    noise_concat= np.concatenate((noise_real, noise_imag), axis=0)
    del noise_real, noise_imag
    
    noisy_concat= np.concatenate((noisy_real, noisy_imag), axis=0)
    del noisy_real, noisy_imag
    
    np.save(path_save_spectrogram + 'voice_concat= '+str(SNR[i]), voice_concat)
    np.save(path_save_spectrogram + 'noise_concat= '+str(SNR[i]), noise_concat)
    np.save(path_save_spectrogram + 'noisy_concat= '+str(SNR[i]), noisy_concat)
    del voice_concat, noisy_concat, noise_concat
    print('SNR= '+str(SNR[i])+' values were saved successfully')

SNR= -10 values were saved successfully
SNR= -5 values were saved successfully
SNR= 0 values were saved successfully
SNR= 5 values were saved successfully
SNR= 10 values were saved successfully
